In [1]:
import time
import torch
import numpy as np
from torch.optim import Adam
from options.TrainOptions import *
from Encoder import *
from Decoder import *
from util import *

# def training(opt, encoder, decoder, train_loader, val_loader):
#     encoder.train()
#     decoder.train()

#     for epoch in range(opt.n_epoch):
#         avg_loss = 0
#         start = time.time()
#         for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(train_loader):

#             utterances = utterances.to(opt.device)
#             labels = labels.to(opt.device)
#             u_lens = u_lens.to(opt.device)
#             l_lens = l_lens.to(opt.device)

#             encoder.zero_grad()
#             decoder.zero_grad()
#             optimizer.zero_grad()

#             keys, values, out_lens = encoder(utterances, u_lens)
#             predict_labels = decoder(keys, values, labels)

#             loss = criterion(predict_labels, labels)
#             mask = torch.arange(labels.size(1)).unsqueeze(0) >= l_lens.unsqueeze(1)

#             loss.backward()
        # 	# `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        # 	nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        # 	avg_loss += loss.item()
        # 	if batch_idx % 10 == 9:
        # 		print('batch', batch_idx + 1, 'Loss', avg_loss / 10, 'Running time', time.time() - start)
        # 		avg_loss = 0

        # 	optimizer.step()

        # torch.save(model.state_dict(), './model_{}.pt'.format(epoch))
        # validation(model, val_loader)


opt = TrainOptions().parser.parse_args(args = [])
opt.device = 'cpu'
speech_train = np.load(opt.dataroot + 'train_new.npy', allow_pickle=True, encoding='bytes')
speech_valid = np.load(opt.dataroot + 'dev_new.npy', allow_pickle=True, encoding='bytes')
# speech_test = np.load(opt.dataroot + 'test.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load(opt.dataroot + 'train_transcripts.npy', allow_pickle=True,encoding='bytes')
transcript_valid = np.load(opt.dataroot + 'dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")
encoder = Encoder(opt)
decoder = Decoder(opt)
encoder.to(opt.device)
decoder.to(opt.device)
optimizer = Adam(list(encoder.parameters()) + list(decoder.parameters()), opt.lr)
criterion = nn.CrossEntropyLoss(reduction = 'none')
criterion.to(opt.device)

transcript_train = transform_letter_to_index(transcript_train)
transcript_valid = transform_letter_to_index(transcript_valid)

print("Transfer the transcript from letters to index sucessfully.....")
train_data = MyDataset(speech_train, transcript_train)
dev_data = MyDataset(speech_valid, transcript_valid)

train_loader_args = dict(batch_size = opt.train_batch_size, pin_memory=True, shuffle = True, collate_fn = collate_fn) 
train_loader = Data.DataLoader(train_data, **train_loader_args)
dev_loader_args = dict(shuffle=False, batch_size = opt.val_batch_size, pin_memory=True, collate_fn = collate_fn) 
dev_loader = Data.DataLoader(dev_data, **dev_loader_args)

# training(opt, encoder, decoder, train_loader, dev_loader)



Data Loading Sucessful.....
Transfer the transcript from letters to index sucessfully.....


In [2]:
encoder.train()
decoder.train()

for epoch in range(opt.n_epoch):
    avg_loss = 0
    start = time.time()
    for batch_idx, (utterances, labels, u_lens, l_lens) in enumerate(train_loader):
        utterances.permute(0, 2, 1)
        utterances = utterances.to(opt.device)
        labels = labels.to(opt.device)
        u_lens = u_lens.to(opt.device)
        l_lens = l_lens.to(opt.device)
        
        encoder.zero_grad()
        decoder.zero_grad()
        optimizer.zero_grad()

        keys, values, out_lens = encoder(utterances, u_lens)
        break
#         predict_labels = decoder(keys, values, labels)

#         loss = criterion(predict_labels, labels)
#         mask = torch.arange(labels.size(1)).unsqueeze(0) >= l_lens.unsqueeze(1)

#         loss.backward()

RuntimeError: start (4096) + length (64) exceeds dimension size (4096).

In [3]:
utterances.shape

torch.Size([64, 1666, 40])

In [2]:
training(opt, encoder, decoder, train_loader, dev_loader)

RuntimeError: start (4096) + length (64) exceeds dimension size (4096).

In [3]:
%debug

> /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/rnn.py(529)forward_impl()
    527         else:
    528             result = _VF.lstm(input, batch_sizes, hx, self._get_flat_weights(), self.bias,
--> 529                               self.num_layers, self.dropout, self.training, self.bidirectional)
    530         output = result[0]
    531         hidden = result[1:]

ipdb> up
> /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/rnn.py(554)forward_packed()
    552         max_batch_size = int(max_batch_size)
    553 
--> 554         output, hidden = self.forward_impl(input, hx, batch_sizes, max_batch_size, sorted_indices)
    555 
    556         output = PackedSequence(output, batch_sizes, sorted_indices, unsorted_indices)

ipdb> up
> /home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/modules/rnn.py(562)forward()
    560     def forward(self, input, hx=None):
    561         if isinstanc